<a href="https://www.kaggle.com/code/akokoo/s4e02-data-xgboost-eda?scriptVersionId=161851073" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# **DATA -1 PRE** 

In [2]:
train_data = pd.read_csv('/kaggle/input/playground-series-s4e2/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e2/test.csv')

##### 训练集

In [3]:
print(train_data.head(3))

   id  Gender        Age    Height     Weight family_history_with_overweight  \
0   0    Male  24.443011  1.699998  81.669950                            yes   
1   1  Female  18.000000  1.560000  57.000000                            yes   
2   2  Female  18.000000  1.711460  50.165754                            yes   

  FAVC      FCVC       NCP        CAEC SMOKE      CH2O SCC       FAF  \
0  yes  2.000000  2.983297   Sometimes    no  2.763573  no  0.000000   
1  yes  2.000000  3.000000  Frequently    no  2.000000  no  1.000000   
2  yes  1.880534  1.411685   Sometimes    no  1.910378  no  0.866045   

        TUE       CALC                 MTRANS           NObeyesdad  
0  0.976473  Sometimes  Public_Transportation  Overweight_Level_II  
1  1.000000         no             Automobile        Normal_Weight  
2  1.673584         no  Public_Transportation  Insufficient_Weight  


##### 查看测试集

In [4]:
print(test_data.head(3))

      id  Gender        Age    Height      Weight  \
0  20758    Male  26.899886  1.848294  120.644178   
1  20759  Female  21.000000  1.600000   66.000000   
2  20760  Female  26.000000  1.643355  111.600553   

  family_history_with_overweight FAVC      FCVC  NCP       CAEC SMOKE  \
0                            yes  yes  2.938616  3.0  Sometimes    no   
1                            yes  yes  2.000000  1.0  Sometimes    no   
2                            yes  yes  3.000000  3.0  Sometimes    no   

       CH2O SCC     FAF       TUE       CALC                 MTRANS  
0  2.825629  no  0.8554  0.000000  Sometimes  Public_Transportation  
1  3.000000  no  1.0000  0.000000  Sometimes  Public_Transportation  
2  2.621877  no  0.0000  0.250502  Sometimes  Public_Transportation  


In [5]:
print(train_data.describe())

                id           Age        Height        Weight          FCVC  \
count  20758.00000  20758.000000  20758.000000  20758.000000  20758.000000   
mean   10378.50000     23.841804      1.700245     87.887768      2.445908   
std     5992.46278      5.688072      0.087312     26.379443      0.533218   
min        0.00000     14.000000      1.450000     39.000000      1.000000   
25%     5189.25000     20.000000      1.631856     66.000000      2.000000   
50%    10378.50000     22.815416      1.700000     84.064875      2.393837   
75%    15567.75000     26.000000      1.762887    111.600553      3.000000   
max    20757.00000     61.000000      1.975663    165.057269      3.000000   

                NCP          CH2O           FAF           TUE  
count  20758.000000  20758.000000  20758.000000  20758.000000  
mean       2.761332      2.029418      0.981747      0.616756  
std        0.705375      0.608467      0.838302      0.602113  
min        1.000000      1.000000      0.

In [6]:
print(train_data['NObeyesdad'].value_counts())  #目标变量分布

NObeyesdad
Obesity_Type_III       4046
Obesity_Type_II        3248
Normal_Weight          3082
Obesity_Type_I         2910
Insufficient_Weight    2523
Overweight_Level_II    2522
Overweight_Level_I     2427
Name: count, dtype: int64


In [7]:
print(train_data.isnull().sum())

id                                0
Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64


In [8]:
print(test_data.describe())

                 id           Age        Height        Weight          FCVC  \
count  13840.000000  13840.000000  13840.000000  13840.000000  13840.000000   
mean   27677.500000     23.952740      1.698934     87.384504      2.442898   
std     3995.408198      5.799814      0.088761     26.111819      0.531606   
min    20758.000000     14.000000      1.450000     39.000000      1.000000   
25%    24217.750000     20.000000      1.631662     65.000000      2.000000   
50%    27677.500000     22.906342      1.700000     83.952968      2.358087   
75%    31137.250000     26.000000      1.760710    111.157811      3.000000   
max    34597.000000     61.000000      1.980000    165.057269      3.000000   

                NCP          CH2O           FAF           TUE  
count  13840.000000  13840.000000  13840.000000  13840.000000  
mean       2.750610      2.032044      0.974532      0.611033  
std        0.710927      0.611230      0.840361      0.608005  
min        1.000000      1.00000

# **DATA -2 EDA**

In [9]:
# 检查列数是否匹配
if train_data.shape[1] == test_data.shape[1]:
    print("列数匹配！")
else:
    print("警告：训练集和测试集的列数不匹配。")


警告：训练集和测试集的列数不匹配。


In [10]:
# 检查训练集和测试集的列
train_columns = set(train_data.columns)
test_columns = set(test_data.columns)

# 找出训练集中有而测试集中没有的列
extra_columns_in_train = train_columns - test_columns

# 找出测试集中有而训练集中没有的列
extra_columns_in_test = test_columns - train_columns

print("训练集中多余的列：", extra_columns_in_train)
print("测试集中多余的列：", extra_columns_in_test)


训练集中多余的列： {'NObeyesdad'}
测试集中多余的列： set()


In [11]:
from sklearn.preprocessing import LabelEncoder
features = train_data.drop(['id', 'NObeyesdad'], axis=1)
target = train_data['NObeyesdad']
# 目标变量标签编码
features_encoded = pd.get_dummies(features)
label_encoder = LabelEncoder()
target_encoded = label_encoder.fit_transform(target)

# 划分训练集和验证集
X_train, X_valid, y_train, y_valid = train_test_split(features_encoded, target_encoded, test_size=0.2, random_state=42)

# 初始化XGBoost分类器
model = XGBClassifier()

# 训练模型
model.fit(X_train, y_train)

# 预测验证集
predictions = model.predict(X_valid)

# 评估模型准确性
accuracy = accuracy_score(y_valid, predictions)
print("模型在验证集的准确性：", accuracy)


模型在验证集的准确性： 0.9019749518304432


In [12]:
class_mapping = {
    0: 'Insufficient_Weight',
    1: 'Normal_Weight',
    2: 'Obesity_Type_I',
    3: 'Obesity_Type_II',
    4: 'Obesity_Type_III',
    5: 'Overweight_Level_I',
    6: 'Overweight_Level_II'
}

# 获取训练集中的独热编码列
encoded_columns = list(features_encoded.columns)

# 在测试集上进行预测
test_features_encoded = pd.get_dummies(test_data.drop(['id'], axis=1))
test_features_encoded = test_features_encoded.reindex(columns=encoded_columns, fill_value=0)
test_predictions = model.predict(test_features_encoded)

# 使用 class_mapping 映射预测结果为字符串标签
submission_df = pd.DataFrame({'id': test_data['id'], 'NObeyesdad': test_predictions})
submission_df['NObeyesdad'] = submission_df['NObeyesdad'].map(class_mapping)

# 生成提交文件
submission_df.to_csv('submission.csv', index=False)


In [13]:
subdata= pd.read_csv('./submission.csv')
print(subdata.head(10))

      id           NObeyesdad
0  20758      Obesity_Type_II
1  20759   Overweight_Level_I
2  20760     Obesity_Type_III
3  20761       Obesity_Type_I
4  20762     Obesity_Type_III
5  20763  Insufficient_Weight
6  20764  Insufficient_Weight
7  20765        Normal_Weight
8  20766  Overweight_Level_II
9  20767        Normal_Weight
